<a href="https://colab.research.google.com/github/rjsdn2308/machine-learning-practice/blob/main/2505221_movie_recommandation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Lens Dataset Recommandation

600명의 사용자가 9,000편의 영화에 10만 건의 평점과 3,600건의 태그를 적용한 데이터셋

- rating (이번 추천 알고리즘에 사용)
- movies
- links
- tags

## 라이브러리 불러오기

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
import numpy as np

## 데이터 불러오기 및 확인

In [5]:
# 평점
ratings = pd.read_csv('/content/drive/MyDrive/2025-1/ml/machine-learning-practice/data/movielens/ratings.csv')
ratings_with_datatime = ratings.copy()
ratings_with_datatime['date'] = pd.to_datetime(ratings_with_datatime['timestamp'], unit='s')
ratings_with_datatime.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/2025-1/ml/machine-learning-practice/data/movielens/ratings.csv'

In [ ]:
tags = pd.read_csv('/content/drive/MyDrive/2025-1/ml/machine-learning-practice/data/movielens/tags.csv')
tags_with_datatime = tags.copy()
tags_with_datatime['date'] = pd.to_datetime(tags_with_datatime['timestamp'], unit='s')
tags_with_datatime.head()

In [ ]:
links = pd.read_csv('/content/drive/MyDrive/2025-1/ml/machine-learning-practice/data/movielens/links.csv')
links.head()

In [ ]:
movies = pd.read_csv('/content/drive/MyDrive/2025-1/ml/machine-learning-practice/data/movielens/movies.csv')
movies.head()

In [ ]:
#movieId = 60756 이거나 89774인 movies data 출력
movie_id = [60756, 89774]
movies[movies['movieId'].isin(movie_id)]

In [ ]:
train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)
train_data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 사용자-아이템 행렬 생성
train_matrix = train_data.pivot(index='userId', columns='movieId', values='rating')
test_matrix = test_data.pivot(index='userId', columns='movieId', values='rating')
train_matrix

### 결측값 처리
- 평점이 없는 경우 0으로 처리

In [ ]:
train_matrix = train_matrix.fillna(0)
test_matrix = test_matrix.fillna(0)
train_matrix

## 사용자 기반 협업 필터링

In [ ]:
# 사용자 유사도
user_similarity = cosine_similarity(train_matrix)  # 사용자 간 코사인 유사도 계산
user_similarity = pd.DataFrame(user_similarity, index=train_matrix.index, columns=train_matrix.index)
user_similarity

In [ ]:
def predict_user_based_cf(user_id, movie_id):
    # 유사도와 평점을 기반으로 예측
    if movie_id not in train_matrix.columns:
        return 0  # 영화가 훈련 데이터에 없으면 0 반환
    similar_users = user_similarity[user_id]
    ratings = train_matrix[movie_id]
    weighted_sum = np.dot(similar_users, ratings)
    similarity_sum = np.sum(similar_users[ratings > 0])
    if similarity_sum == 0:
        return 0
    return weighted_sum / similarity_sum

In [ ]:
user_based_predictions = []
for user_id, movie_id, rating in test_data[['userId', 'movieId', 'rating']].values:
    pred = predict_user_based_cf(user_id, movie_id)
    user_based_predictions.append(pred)

In [ ]:
from re import T
train_matrix[movie_id[1]]
# 이중에서 값이 0이 아닌것만 출력
series = train_matrix[movie_id[1]][train_matrix[movie_id[1]] > 0]
series

user_id 103번과 495는 복싱영화의 평점을 높게 평가

In [ ]:
user_id = 103 # 사용자 ID 103번
pred_movie_id = movie_id[0]  # 영화 ID 60756
predicted_rating = predict_user_based_cf(user_id, pred_movie_id)

print(f"{user_id}번 사용자의 영화번호 {pred_movie_id}번의 평점 예측: {predicted_rating:.2f}")

In [ ]:
user_based_rmse = np.sqrt(mean_squared_error(test_data['rating'], user_based_predictions))
print(f"사용자 기반 협업 필터링 RMSE: {user_based_rmse:.4f}")

## 아이템 기반 협업 필터링

In [ ]:
item_similarity = cosine_similarity(train_matrix.T)  # 아이템 간 코사인 유사도 계산
item_similarity = pd.DataFrame(item_similarity, index=train_matrix.columns, columns=train_matrix.columns)
item_similarity

In [ ]:
def predict_item_based(user_id, movie_id):
    # 유사도와 평점을 기반으로 예측
    if movie_id not in train_matrix.columns:
        return 0  # 영화가 훈련 데이터에 없으면 0 반환
    similar_items = item_similarity[movie_id]
    user_ratings = train_matrix.loc[user_id]
    weighted_sum = np.dot(similar_items, user_ratings)
    similarity_sum = np.sum(similar_items[user_ratings > 0])
    if similarity_sum == 0:
        return 0
    return weighted_sum / similarity_sum

In [ ]:
train_matrix.loc[user_id]

In [ ]:
item_based_predictions = []
for user_id, movie_id, rating in test_data[['userId', 'movieId', 'rating']].values:
    pred = predict_item_based(user_id, pred_movie_id)
    item_based_predictions.append(pred)

In [ ]:
print(f"{user_id}번 사용자의 영화번호 {pred_movie_id}번의 평점 예측: {predicted_rating:.2f}")

In [ ]:
item_based_rmse = np.sqrt(mean_squared_error(test_data['rating'], item_based_predictions))
print(f"아이템 기반 협업 필터링 RMSE: {item_based_rmse:.4f}")

## 성능 비교

In [ ]:
print(f"사용자 기반 협업 필터링 RMSE: {user_based_rmse:.4f}")
print(f"아이템 기반 협업 필터링 RMSE: {item_based_rmse:.4f}")

확인문제 1 : 60756번 영화를 본 사용자 중에 1명을 찾아서 해당 사용자의 89774번 영화의 평점을 예측하세요

확인문제 2 : 위 성능 비교 했을 때 사용자 기반과 아이템 기반 중 더 좋은 성능을 나타내는 방법을 쓰세요
- 사용자 기반보다 아이템 기반이 더 평점 예측이 더 정확하다고 볼 수 있습니다. 그 이유는 아이템 기반 협업 필터링이 사용자 기반 협업 필터링보다 평점 예측에서 더 정확하기 때문입니다.


In [ ]:
user_id = 1
pred_movie_id = movie_id[0]  # 영화 ID 60756
predicted_rating = predict_user_based_cf(user_id, pred_movie_id)

print(f"{user_id}번 사용자의 영화번호 {pred_movie_id}번의 평점 예측: {predicted_rating:.2f}")